In [64]:
import pandas as pd
import numpy as np
from functools import partial
from tqdm import tqdm_notebook
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import mean_squared_error
%matplotlib inline
pd.set_option('display.max_columns', 500)

#train
train = pd.read_csv('input/train2.csv')
# df.drop(['start_square', 'plan_s', 'plan_m', 'plan_l', 'vid_0', 'vid_1', 'vid_2'], axis=1, inplace=True)
# df.to_csv("./input/train_full.csv", index=False)

#test
test = pd.read_csv('input/test2.csv')

K = 3

kf = KFold(n_splits=K, random_state=1337, shuffle=True)
kf.get_n_splits(range(len(train)))

print(kf)

KFold(n_splits=3, random_state=1337, shuffle=True)


In [41]:
list(enumerate(pd.read_csv('input/train2.csv').columns))

[(0, 'id'),
 (1, 'bulk_id'),
 (2, 'spalen'),
 (3, 'date1'),
 (4, 'value'),
 (5, 'month'),
 (6, 'month_cnt'),
 (7, 'Класс объекта'),
 (8, 'Количество помещений'),
 (9, 'Огорожена территория'),
 (10, 'Площадь земельного участка'),
 (11, 'Входные группы'),
 (12, 'Детский сад'),
 (13, 'Школа'),
 (14, 'Поликлиника'),
 (15, 'ФОК'),
 (16, 'Спортивная площадка'),
 (17, 'Автомойка'),
 (18, 'Кладовые'),
 (19, 'Колясочные'),
 (20, 'Кондиционирование'),
 (21, 'Вентлияция'),
 (22, 'Лифт'),
 (23, 'Система мусоротведения'),
 (24, 'Видеонаблюдение'),
 (25, 'Подземная парковка'),
 (26, 'Двор без машин'),
 (27, 'Машиномест'),
 (28, 'Площадь пром. зоны в радиусе 500 м'),
 (29, 'Площадь зеленой зоны в радиусе 500 м'),
 (30, 'До Кремля'),
 (31, 'До ТТК(км)'),
 (32, 'До Садового(км)'),
 (33, 'До большой дороги на машине(км)'),
 (34, 'До удобной авторазвязки на машине(км)'),
 (35, 'До метро пешком(км)'),
 (36, 'До промки(км)'),
 (37, 'До парка(км)'),
 (38, 'До парка пешком(км)'),
 (39, 'Станций метро от коль

In [65]:
for i, (train_index, valid_index) in enumerate(kf.split(range(len(train)), train['bulk_id'])):
    train.loc[train_index].sort_values(['bulk_id', 'spalen', 'date1']).to_csv(f"input/train_{i}.csv",index=False)
    train.loc[valid_index].sort_values(['bulk_id', 'spalen', 'date1']).to_csv(f"input/valid_{i}.csv",index=False)

In [211]:
from catboost import CatBoostRegressor, Pool

TEST_FILE = 'input/test2.csv' #add
CD_FILE = 'input/train.cd'

test_pool = Pool(TEST_FILE, column_description=CD_FILE, has_header=True, delimiter=",")

oof = pd.DataFrame({'value': None}, index=train['id'])

In [48]:
importance = model.get_feature_importance(val_pool)

In [57]:
[train_pool.get_feature_names()[i] for i in np.argsort(importance)[::-1]]

['flats_left_reserved',
 'sum_sq_reserved',
 'value_lag_reserved',
 'value_lag_active',
 'flats_left_active',
 'sum_sq_active',
 'flats_sold_lag_active',
 'До парка(км)',
 'spalen',
 'До метро пешком(км)',
 'flats_sold_lag_reserved',
 'bulk_id',
 'month',
 'Cтавка по ипотеке',
 'min_price_reserved',
 'До парка пешком(км)',
 'До большой дороги на машине(км)',
 'Машиномест',
 'min_price_active',
 'min_sq_reserved',
 'Площадь двора',
 'Вклады свыше 3 лет',
 'Вклады до 1 года',
 'mean_sq_active',
 'Видеонаблюдение',
 'Класс объекта',
 'max_price_active',
 'max_sq_reserved',
 'Поликлиника',
 'mean_price_reserved',
 'mean_price_active',
 'Вентлияция',
 'max_sq_active',
 'Кладовые',
 'mean_fl_reserved',
 'max_fl_reserved',
 'max_price_reserved',
 'max_fl_active',
 'mean_fl_active',
 'Вклады от 1 года до 3 лет',
 'mean_sq_reserved',
 'min_sq_active',
 'Детский сад',
 'Количество помещений',
 'Курс',
 'До удобной авторазвязки на машине(км)',
 'ФОК',
 'Площадь земельного участка',
 'Школа',
 'Пл

In [212]:
for i in range(K):

    TRAIN_FILE = f"input/train_{i}.csv"
    VAL_FILE = f"input/valid_{i}.csv"

    train_pool = Pool(TRAIN_FILE, column_description=CD_FILE, has_header=True, delimiter=",")
    val_pool = Pool(VAL_FILE, column_description=CD_FILE, has_header=True, delimiter=",")

    model = CatBoostRegressor(iterations=1000,
                              #n_estimators=1000,
                              learning_rate=0.01,
                              depth=10,
                              #l2_leaf_reg=10,
                              random_seed=1337)
    
    #model = model.load_model(f"models/value_{i}")
    
    model.fit(train_pool, eval_set=val_pool, verbose=500)
    model.save_model(f"models/value_no_reserved_{i}")
    
#     pred = model.predict(test_pool)
#     test[f"flats_sold_{i}"] = pred
 
#     pred = model.predict(val_pool)
#     val = pd.read_csv(VAL_FILE, usecols=['id'], index_col='id')
#     oof.loc[val.index, 'flats_sold'] = pred
    

0:	learn: 423.9898431	test: 436.0666276	best: 436.0666276 (0)	total: 59.5ms	remaining: 59.4s
500:	learn: 204.1590529	test: 224.2077593	best: 224.2077593 (500)	total: 41s	remaining: 40.8s

bestTest = 206.3329246
bestIteration = 999

Shrink model to first 1000 iterations.
0:	learn: 421.0218818	test: 441.3710680	best: 441.3710680 (0)	total: 142ms	remaining: 2m 21s
500:	learn: 189.5361221	test: 243.0647081	best: 243.0647081 (500)	total: 1m 9s	remaining: 1m 9s

bestTest = 233.9327484
bestIteration = 999

Shrink model to first 1000 iterations.
0:	learn: 438.5332680	test: 406.0139134	best: 406.0139134 (0)	total: 117ms	remaining: 1m 56s
500:	learn: 198.3294256	test: 219.2711398	best: 219.2711398 (500)	total: 1m 7s	remaining: 1m 7s

bestTest = 213.005561
bestIteration = 999

Shrink model to first 1000 iterations.


In [68]:
from datetime import datetime as dt

In [69]:
test_dates = test['date1'].apply(dt.fromtimestamp).unique()

In [130]:
flats_train = pd.read_csv('output/flat_train_predict.csv', 
                    usecols=['bulk_id', 'spalen', 'id_flatwork', 'floor', 'square', 'pricem2', 'rank'],
                    index_col=['bulk_id', 'spalen', 'id_flatwork'])

flats_test = pd.read_csv('output/flat_test_predict.csv', 
                    usecols=['bulk_id', 'spalen', 'id_flatwork', 'floor', 'square', 'pricem2', 'rank'],
                    index_col=['bulk_id', 'spalen', 'id_flatwork'])

flats = pd.concat([flats_train, flats_test]).groupby(level=[0, 1], group_keys=False) \
    .apply(lambda df: df.sort_values('rank')).join(flat)

In [131]:
flat_active = pd.read_csv('output/flat_active.csv', dtype={'spalen': int},
                         index_col=['date1', 'bulk_id', 'spalen', 'id_flatwork'], parse_dates=['date1']).sort_index()

flat_reserved = pd.read_csv('output/flat_reserved.csv', dtype={'spalen': int},
                         index_col=['date1', 'bulk_id', 'spalen', 'id_flatwork'], parse_dates=['date1']).sort_index()

In [132]:
flat_active =flat_active.join(flats.reset_index(['bulk_id', 'spalen'])['rank'])
flat_reserved = flat_reserved.join(flats.reset_index(['bulk_id', 'spalen'])['rank'])

In [122]:
def recalculate_features(df, flats=None, date=None):
    idx = (df['bulk_id'].iloc[0], df['spalen'].iloc[0])
    
    prev_date = (pd.to_datetime(date) - pd.offsets.MonthBegin()).to_datetime64()
    
    def get_flats_left(flat_data):
        available_flats = flat_data.loc[date]
        prev_available_flats = flat_data.loc[prev_date]
        
        if available_flats.index.contains(idx):
            available_flats = available_flats.loc[idx]
            if prev_available_flats.index.contains(idx):
                prev_available_flats = prev_available_flats.loc[idx]
                non_available_index = prev_available_flats.index.difference(available_flats.index)
                
                non_available_flats = prev_available_flats.loc[non_available_index]
            else:
                non_available_flats = prev_available_flats.iloc[:0]
        else:
            available_flats = available_flats.iloc[:0]
            if prev_available_flats.index.contains(idx):
                non_available_flats = prev_available_flats.loc[idx]
            else:
                non_available_flats = prev_available_flats.iloc[:0]
            
        if flats.index.contains(idx) :
            flats_left = flats.loc[idx]

            flats_left = flats_left.loc[flats_left.index.intersection(available_flats.index).difference(non_available_flats.index)]

            flats_left = pd.concat([flats_left, non_available_flats])

        else:
            flats_left = non_available_flats
            
        flats_left.sort_values('rank', inplace=True)
            
        return flats_left
    
    def get_data(flat_data):
        
        flats_left = get_flats_left(flat_data)
            
        data_left = flats_left[flats_left['square'].cumsum() > df['value_predict_active'].iloc[0]]
        data_sold = flats_left[flats_left['square'].cumsum() <= df['value_predict_active'].iloc[0]]
            
        return {'flats_left': len(data_left),
                'max_fl': data_left['floor'].max(), 
                'min_fl': data_left['floor'].min(),
                'mean_fl': data_left['floor'].mean(),
                'max_sq': data_left['square'].max(), 
                'min_sq': data_left['square'].min(),
                'mean_sq': data_left['square'].mean(),
                'sum_sq': data_left['square'].sum(),
                'max_price': data_left['pricem2'].max(),
                'min_price': data_left['pricem2'].min(),
                'mean_price': data_left['pricem2'].mean(),
                'value_predict': data_sold['square'].sum()}
    
    data_active = get_data(flat_active)
    data_reserved = get_data(flat_reserved)
    
    result = {k + '_active': v for k, v in data_active.items()}
    result.update({k + '_reserved': v for k, v in data_reserved.items()})
    
    return pd.Series(result)

def drop_sold_flats(flats_left, prediction, date):
    flats_left_idx = []

    prev_date = (pd.to_datetime(date) - pd.offsets.MonthBegin()).to_datetime64()
    
    def get_flats_left(flat_data, idx):
        available_flats = flat_data.loc[date]
        prev_available_flats = flat_data.loc[prev_date]
        
        if available_flats.index.contains(idx):
            available_flats = available_flats.loc[idx]
            if prev_available_flats.index.contains(idx):
                prev_available_flats = prev_available_flats.loc[idx]
                non_available_index = prev_available_flats.index.difference(available_flats.index)
                
                non_available_flats = prev_available_flats.loc[non_available_index]
            else:
                non_available_flats = prev_available_flats.iloc[:0]
        else:
            available_flats = available_flats.iloc[:0]
            if prev_available_flats.index.contains(idx):
                non_available_flats = prev_available_flats.loc[idx]
            else:
                non_available_flats = prev_available_flats.iloc[:0]
            
        if flats.index.contains(idx) :
            flats_left = flats.loc[idx]

            flats_left = flats_left.loc[flats_left.index.intersection(available_flats.index).difference(non_available_flats.index)]

            flats_left = pd.concat([flats_left, non_available_flats])

        else:
            flats_left = non_available_flats
            
        flats_left.sort_values('rank', inplace=True)
            
        return flats_left
    

    for idx, g in flats_left.groupby(level=[0, 1]).groups.items():

        df = get_flats_left(flat_active, idx)
        
        if prediction.index.contains(idx):     
            df = df[df['square'].cumsum() > prediction.loc[idx]]
            
        flats_left_idx.append(pd.concat([df], keys=[idx], names=['id_bulk', 'spalen']))

    return pd.concat(flats_left_idx)

In [213]:
flats_left = flats[flats['sale'] >= test_dates[0]]

next_flats_left = None
next_date_features = None
prev_test_date = None

test_predict_date = []

for date in tqdm_notebook(test_dates):

    test_date_file = f"output/test_date_{date}.csv"
    test_date = test[test['date1'].apply(dt.fromtimestamp) == date].copy()
    
    if next_flats_left is not None:
        flats_left = next_flats_left
    
    if next_date_features is not None:
        cols = [c + suffix for c in ['flats_left', 'mean_fl', 'mean_sq', 'max_fl', 'max_sq', 
                                     'min_fl', 'min_sq', 'sum_sq',
                                     'mean_price', 'max_price', 'min_price'] for suffix in ['_active', '_reserved']]
        test_date = pd.merge(test_date.drop(cols, axis=1),
                             next_date_features.reset_index(), on=['bulk_id', 'spalen'], how='left')
    if prev_test_date is not None:
        test_date = pd.merge(test_date.drop([c + suffix for c in ['flats_sold_lag', 'value_lag'] for suffix in ['_active', '_reserved']], axis=1, errors='ignore'),
                             prev_test_date.drop([c + suffix for c in ['flats_sold_lag', 'value_lag'] for suffix in ['_active', '_reserved']], axis=1)[['bulk_id', 'spalen', 'flats_sold_active', 'value_predict_active', 'flats_sold_reserved', 'value_predict_reserved']]
                                 .rename(columns={'flats_sold_active': 'flats_sold_lag_active', 
                                                  'value_predict_active': 'value_lag_active',
                                                  'flats_sold_reserved': 'flats_sold_lag_reserved', 
                                                  'value_predict_reserved': 'value_lag_reserved',}), on=['bulk_id', 'spalen'], how='left') \
            .fillna(0)
    test_date.to_csv(test_date_file, index=False, columns=test.columns)
    
    test_pool = Pool(test_date_file, column_description=CD_FILE, has_header=True, delimiter=",")

    test_date['value_predict_active'] = 0
    
    for i in range(K):
        model = CatBoostRegressor().load_model(f"models/value_no_reserved_{i}")
        test_date['value_predict_active'] = test_date['value_predict_active'] + model.predict(test_pool)
        
    test_date['value_predict_active'] = test_date['value_predict_active'] / K

    prediction = test_date.set_index(['bulk_id', 'spalen'])['value_predict_active'].sort_index()

    print('Recalculating features...')
    next_date_features = test_date.groupby(['bulk_id', 'spalen']) \
        .apply(partial(recalculate_features, flats=flats_left, date=date)) \
        .fillna(0)
    
    #test_date['value_predict_reserved'] = test_date['value_predict_active']
    
    test_date = pd.merge(test_date.drop(['value_predict_reserved'], axis=1, errors='ignore'),
                         next_date_features[['value_predict_reserved']].reset_index(), on=['bulk_id', 'spalen'], how='left')

    print('Dropping flats...')
    #next_flats_left = drop_sold_flats(flats_left, prediction, date)

    
    prev_test_date = test_date.copy()
    
    test_date['date1'] = test_date['date1'].apply(dt.fromtimestamp)
    test_predict_date.append(test_date)

Recalculating features...
Dropping flats...
Recalculating features...
Dropping flats...
Recalculating features...
Dropping flats...


In [214]:
next_date_features['date1'] = pd.to_datetime(test_dates[-1]) + pd.offsets.MonthBegin(normalize=True)

In [215]:
columns = ['bulk_id', 'spalen', 'date1', 'sum_sq_active', 'value', 'value_predict_active']
test_predict = pd.concat(test_predict_date + [next_date_features.reset_index()])[columns].set_index(['bulk_id', 'spalen', 'date1']).sort_index()
test_predict.index.set_levels(test_predict.index.levels[2].map(lambda x: int(x.timestamp())), level=2, inplace=True)

In [216]:
df = pd.merge(test[['bulk_id', 'spalen', 'date1', 'value']], 
              test_predict.reset_index().drop('value', axis=1), on=['bulk_id', 'spalen', 'date1'], how='left')

#df = df[df['date1'].apply(dt.fromtimestamp) == test_dates[0]]
#np.sqrt(mean_squared_error(df['value'], df['value_predict']))

In [217]:
submission = pd.merge(test[['id', 'bulk_id', 'spalen', 'date1']], 
         df[['bulk_id', 'spalen', 'date1', 'value_predict_active']], on=['bulk_id', 'spalen', 'date1'], how='left') \
    .rename(columns={'value_predict_active': 'value'})[['id', 'value']]


In [218]:
submission.to_csv('output/submission_final_10.csv', index=False)

In [228]:
model1 = pd.read_csv('output/sample5.csv', index_col=['id'])

In [229]:
model2 = pd.read_csv('output/submission_final_6.csv', index_col=['id'])

In [230]:
model3 = pd.read_csv('output/submission_final_10.csv', index_col=['id'])

In [253]:
model4 = pd.read_csv('/Users/nmurzin/Downloads/sample9.csv', index_col=['id'])

In [254]:
df = model1.join(model2, lsuffix='_1', rsuffix='_2').join(model4, rsuffix='_3')

In [250]:
df['value_1'] = 0.6 * df['value_1'] 
df['value_2'] = 0.4 * df['value_2'] 
#df['value'] = 0.333 * df['value'] 

In [255]:
df['value'] = df[['value_1', 'value_2', 'value']].mean(axis=1)

In [256]:
df['value'].reset_index().to_csv('output/submission_mean_weighted_8.csv', index=False)